# Preparando el dataset
Utilizaremos FairFace como dataset. Este posee las caracteristicas de que tiene igual proporcion en todas las etnias, a saber, Medio Oriente, Blancos, Negros, Hispanos-Latinos, Indios, Sur de Asia y Este de Asia. Al tener una buena proporcion de las etnias que generalmente no estan correctamente representadas no existira un bias significativo en el modelo. Las fotos han sido cropped por lo que este paso no es necesario. Debemos tener en cuenta que de procesar fotos nuevas estas tambien tiene que ser cropped utilizando dlib o alguna otra herramienta similar.

No nos hara falta las fotos del Medio Oriente por lo que podemos desecharlas.

In [15]:
import pandas

train_csv = pandas.read_csv('./Datasets/FairFace/fairface_label_train.csv')
test_csv = pandas.read_csv('./Datasets/FairFace/fairface_label_val.csv')

train_csv = train_csv[train_csv['race'] != 'Middle Eastern']
test_csv = test_csv[test_csv['race'] != 'Middle Eastern']

Mezclamos las dos etnias asiaticas en una, ya que no se nos exige distinguirlas.

In [20]:
indexs_train = train_csv[(train_csv['race'] == 'East Asian') | (train_csv['race'] == 'Southeast Asian')].index

train_csv.loc[indexs_train, 'race'] = 'Asian'
 
indexs_test = test_csv[(test_csv['race'] == 'East Asian') | (test_csv['race'] == 'Southeast Asian')].index

test_csv.loc[indexs_test, 'race'] = 'Asian'


# Convirtiendo las imagenes a feature vectors
Una vez tenemos el dataset preparado procedemos a convertir las imagenes a vectores de caracteristicas que sirvan de input para la red. Se carga la imagen en rgb y se resize a la size que se pide, se transforma esta en una matriz de 21x28 con vectores de longitud 3 por casilla, indicando cada valor en este vector el valor correspondiente al pixel en cada uno de los colores. Posteriormente se normalizan.

In [52]:
import keras.preprocessing.image as image

def getImagePixels(file):
    img = image.load_img(file, target_size=(21, 28,))
    return image.img_to_array(img)

train_features = []
for _, value in train_csv['file'].items():
    train_features.append(getImagePixels('./Datasets/FairFace/' + value)/255)

test_features = []
for _, value in test_csv['file'].items():
    test_features.append(getImagePixels('./Datasets/FairFace/' + value)/255)



# Labels encoding
Una vez listos los features vectors debemos codificar los labels en vectores one hot para el entrenamiento.

In [ ]:
train_labels = []
test_labels = []

